In [ ]:
import xarray as xr
import numpy as np
import hvplot.xarray
from rasterio.warp import transform

In [ ]:
def get_latlon(da):
    
    # Compute the lon/lat coordinates with rasterio.warp.transform
    ny, nx = len(da['y']), len(da['x'])
    x, y = np.meshgrid(da['x'], da['y'])

    # Rasterio works with 1D arrays
    lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
                         x.flatten(), y.flatten())
    lon = np.asarray(lon).reshape((ny, nx))
    lat = np.asarray(lat).reshape((ny, nx))
    da.coords['lon'] = (('y', 'x'), lon)
    da.coords['lat'] = (('y', 'x'), lat)
    da = da.drop_vars(["x", "y"])
    return da

fn = "../data/onwind-av-595.tif"

def convert_to_dataset(fn):
    da = xr.open_rasterio(fn)
    da = get_latlon(da).mean(dim='band')
    ds = da.where((da.values < 255) & (da.values > 0)).to_dataset(name='availability')
    #ds.to_netcdf(fn.replace(".tif", ".nc")) # much memory
    return ds


%memit
ds = convert_to_dataset(fn)

%memit
ds.hvplot.contourf(
    'lon',
    'lat',
    'availability',
    geo=True,
    tiles="CartoLight",
    cmap='Greens',
    alpha=0.5,
    frame_height=800,
    colorbar=False,
    legend=False,
    hover=False,
    title="Available Land"
).opts(
    active_tools=['pan', 'wheel_zoom']
)